In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../data2/Sdf16_1a.txt', sep='\t')
df.head()

,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,...,FL_61V,FL_66V,FL_W01,FL_W31,FL_W61,FL_V95,FL_V02,FL_K14,FL_CE1,FL_CE2
0,100002,N,1,01073,142,13820,Alabama Youth Services,Alabama,AL,N,...,M,M,M,M,M,M,M,M,M,M
1,100005,01504840100000,1,01095,290,10700,Albertville City,Alabama,AL,03,...,M,M,R,R,R,M,M,M,M,M
2,100006,01504800100000,1,01095,290,10700,Marshall County,Alabama,AL,03,...,M,M,R,R,R,M,M,M,M,M
3,100007,01503740100000,1,01073,142,13820,Hoover City,Alabama,AL,03,...,M,M,R,R,R,M,M,M,M,M
4,100008,01504530100000,1,01089,290,26620,Madison City,Alabama,AL,03,...,M,M,R,R,R,M,M,M,M,M


In [4]:
language_df = pd.read_csv('../data2/rla-achievement-lea-sy2016-17.csv')
language_df = language_df[language_df['ALL_RLA00PCTPROF_1617'].str.contains('PS') == False]
language_df.head()

,STNAM,FIPST,LEAID,ST_LEAID,LEANM,DATE_CUR,ALL_RLA00NUMVALID_1617,ALL_RLA00PCTPROF_1617,MAM_RLA00NUMVALID_1617,MAM_RLA00PCTPROF_1617,...,MIG_RLA05NUMVALID_1617,MIG_RLA05PCTPROF_1617,MIG_RLA06NUMVALID_1617,MIG_RLA06PCTPROF_1617,MIG_RLA07NUMVALID_1617,MIG_RLA07PCTPROF_1617,MIG_RLA08NUMVALID_1617,MIG_RLA08PCTPROF_1617,MIG_RLAHSNUMVALID_1617,MIG_RLAHSPCTPROF_1617
0,ALABAMA,1,100005,AL-101,Albertville City,12APR18,2797,28,3,PS,...,22,LE20,22,LE20,18,21-39,14,LT50,20,LE20
1,ALABAMA,1,100006,AL-048,Marshall County,12APR18,2969,34,11,GE50,...,5,PS,4,PS,1,PS,3,PS,3,PS
2,ALABAMA,1,100007,AL-158,Hoover City,12APR18,7421,58,8,LT50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALABAMA,1,100008,AL-169,Madison City,12APR18,5578,69,33,60-69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALABAMA,1,100011,AL-167,Leeds City,12APR18,997,33,3,PS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def parseScore(sc):
    try:
         return int(sc)
    except:
        if '-' in sc:
            x = sc.split('-')
        elif 'LE' in sc or 'LT' in sc:
            x = [0, sc[2:]]
        else:
            x = [100, sc[2:]]
        return (int(x[0]) + int(x[1])) // 2
language_df['ParsedScores'] = language_df['ALL_RLA00PCTPROF_1617'].apply(parseScore)

In [6]:
budget = np.sum(df['TFEDREV'].values)
cut = int(budget * 0.15)
# print('Total Budget : ' + str(budget))
# print('15% of Total Budget : ' + str(cut))
cutting_df = pd.merge(df[['LEAID', 'TFEDREV']], language_df[['LEAID', 'ParsedScores']], on='LEAID')
cutting_df = cutting_df[cutting_df['TFEDREV'] > 0]
# print('Total Districts : ' + str(cutting_df.shape[0]))
top = cutting_df[cutting_df['ParsedScores'] > \
                   (np.mean(cutting_df['ParsedScores']) - np.std(cutting_df['ParsedScores']))]
# print('Districts with percentage of proficent students greater than 1 standard diviation less than from mean : '  \
#       + str(top.shape[0]))
top['Rate'] = (top['TFEDREV'] * 0.25) + (top['ParsedScores'] * 0.75)
total = np.sum(top['Rate'])
top['Normalized'] = top['Rate'] / total
top['BudgetCut'] = round(top['Normalized'] * cut, 2)
top['NewBudget'] = top['TFEDREV'] - top['BudgetCut']

# Problem 1
#### For the districts you selected for budget cuts in HW 3 Problem 4, calculate and visualize the proportion of each district’s total funding that will be lost. Which districts will be affected by your budget cuts the most?

In [8]:
top['BudgetCutProp'] = top['BudgetCut'] / top['TFEDREV'] 
budget_cuts = pd.merge(df, top, on='LEAID', how='left')
budget_cuts['BudgetCutProp'] = budget_cuts['BudgetCutProp'].apply(lambda x: 0 if pd.isna(x) else x)
budget_cuts['InvBudgetCutProp'] = 1 - budget_cuts['BudgetCutProp']
budget_cuts = budget_cuts.sort_values(by='BudgetCutProp', ascending=False)
budget_cuts.head()

,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,...,FL_CE1,FL_CE2,TFEDREV_y,ParsedScores,Rate,Normalized,BudgetCut,NewBudget,BudgetCutProp,InvBudgetCutProp
6794,2600121,N,26,26161,220,11460,Honey Creek Community School,Michigan,MI,01,...,R,R,2000.0,77.0,557.75,1.317457e-07,1098.81,901.19,0.549405,0.450595
7027,2601042,N,26,26163,220,19820,Branch Line School,Michigan,MI,01,...,R,R,1000.0,37.0,277.75,6.560713e-08,547.19,452.81,0.547190,0.452810
9157,3004920,27505640230200,30,30111,N,13740,Broadview H S,Montana,MT,02,...,R,R,3000.0,75.0,806.25,1.904437e-07,1588.38,1411.62,0.529460,0.470540
577,400771,N,4,04019,536,46060,Blue Adobe Project,Arizona,AZ,02,...,M,M,4000.0,75.0,1056.25,2.494960e-07,2080.90,1919.10,0.520225,0.479775
698,400902,N,4,04013,N,38060,The Farm at Mission Montessori Academy,Arizona,AZ,01,...,M,M,4000.0,74.0,1055.50,2.493189e-07,2079.42,1920.58,0.519855,0.480145
